In [87]:
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab

In [88]:
#pip install folium
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab

## Part 1

### *Loading Libraries

In [89]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### *Loading data from Wikipedia

In [90]:
#request = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
#Toronto  = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',header=0)[0]
#Toronto.rename(columns = {'Postcode':'PostalCode'},inplace=True)
Toronto = pd.read_html(io="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M",header=0, na_values=['Not assigned'])[0]


#### * If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [91]:
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
Toronto.Neighbourhood.fillna(Toronto.Borough, inplace=True)
#ignore/remove dataframe rows where Borough = 'Not assigned'
Toronto.dropna(inplace=True)
Toronto.shape

(211, 3)

#### *The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [92]:
Toronto.rename(columns = {'Postcode':'PostalCode'},inplace=True)
Toronto.head(20)

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


#### *More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [93]:
Toronto = Toronto.groupby(['PostalCode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
#Toronto = Toronto.groupby(['PostCode','Borough'],as_index=False).agg(lambda s: ', '.join(s)) 
Toronto.head(20)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


##### Check the table to make sure all neighborhoods with the same zip code are merged together (ex: M5A and M9B) and Queen's Park is shown correctly at Neighbourhood.

In [94]:
## Table checking
Toronto.loc[Toronto['PostalCode']=='M5A']

,PostalCode,Borough,Neighbourhood
53,M5A,Downtown Toronto,"Harbourfront, Regent Park"


In [95]:
## Table checking
Toronto.loc[Toronto['PostalCode']=='M9B']

,PostalCode,Borough,Neighbourhood
94,M9B,Etobicoke,"Cloverdale, Islington, Martin Grove, Princess ..."


In [96]:
# Table checking
Toronto.loc[Toronto['Neighbourhood']=="Queen's Park"]

,PostalCode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


##### Note: Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.

### In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [97]:
Toronto.shape

(103, 3)

## Part 2:  Google Maps Geocoding API Imported

### Get the Latitude,Longitude data for Postcodes

In [98]:
#import lat long data from csv file
GeoInfo = pd.read_csv('http://cocl.us/Geospatial_data')
#uncomment below to check dataframe format 
GeoInfo.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [99]:
#join dataframes
TorontoGeo = Toronto.join(GeoInfo.set_index('Postal Code'), on='PostalCode')
TorontoGeo.head(10)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [100]:
#confirm the dataframe shape still contains the 103 rows
TorontoGeo.shape

(103, 5)

### Use geopy library to get the latitude and longitude values of Toronto.

In [101]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
address = 'Toronto'
geolocator = Nominatim(user_agent="JAC")
location = geolocator.geocode(address)
lat = location.latitude
lon = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(lat, lon))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


## Part 3: Data Visualization - Cluster Toronto Neighbourhoods and load folium library

In [102]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
print('Libraries imported.') 

Libraries imported.


### create map of Toronto postcodes using latitude and longitude values

In [103]:
map_toronto = folium.Map(location=[lat, lon], zoom_start=10)

# add markers to map
for lat, lon, borough, neighbourhood in zip(TorontoGeo['Latitude'], TorontoGeo['Longitude'], TorontoGeo['Borough'], TorontoGeo['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto